In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Brightway imports
import bw2analyzer as ba
import bw2calc as bc
import bw2data as bd
import bw2io as bi
import brightway2 as bw

In [ ]:
import pandas as pd
import numpy as np
import datetime
import os

In [ ]:
# Custom functions
from useful_functions import get_inventory_dataset, init_simple_lca, multi_lcia, multi_contribution_analysis, calculate_projected_impacts
from visualisation_functions import plot_multilca_impacts, plot_contribution_analysis, plot_production_impacts, plot_incremental_impacts, plot_iwplus_contributions, plot_scenario_production_comparison

# Set projects, LCI and LCIA methods

In [ ]:
BW_PROJECT = 'regioinvent' # insert your project name here
bd.projects.set_current(BW_PROJECT)
bd.databases

In [ ]:
EI_DB = 'ecoinvent-3.9.1-cutoff'
LIB_RM_DB = 'LIB raw materials'

## Import LCI

In [ ]:
# (mineral name: activity name, reference product, location)
INVENTORIES = {
    
    ## Neodymium
    "Neodymium":        ("rare earth oxides production, from rare earth carbonate concentrate", "neodymium oxide", "RoW"),
    
    ## Copper
    "Copper concentrate, Canada": ('copper mine operation and beneficiation, sulfide ore', 'copper concentrate, sulfide ore', 'CA'),
    "Market for copper, cathode": ('market for copper, cathode', 'copper, cathode', 'GLO'),
    
    # From Istrate et al (2024)
    "Lithium hydroxide, brine":        ("lithium hydroxide production, Salar de Atacama", "lithium hydroxide, battery grade", "CL"),
    "Lithium hydroxide, spodumene":    ("lithium hydroxide production, from Australian spodumene", "lithium hydroxide, battery grade", "CN"),
    "Cobalt":                ("cobalt sulfate production, from copper-cobalt ore, economic allocation", "cobalt sulfate", "CN"),
    "Nickel":                ("nickel sulfate production, average excluding China, economic allocation", "nickel sulfate", "GLO"),
    "Graphite, natural":     ("natural graphite production, battery grade, from Heilongjiang", "natural graphite, battery grade", "CN"),
    "Graphite, synthetic":   ("graphite powder coating", "synthetic graphite, battery grade", "CN"),
    
    "Lithium carbonate, brine":        ("lithium carbonate production, Salar de Atacama", "lithium carbonate, battery grade", "CL"),
    "Lithium carbonate, spodumene":    ("lithium carbonate production, from Australian spodumene", "lithium carbonate, battery grade", "CN"),
}

In [ ]:
INVENTORIES_ds = get_inventory_dataset(INVENTORIES, database_names=[EI_DB, LIB_RM_DB])

In [ ]:
INVENTORIES_ds

## Pick LCIA methods

In [ ]:
# We can also import some from IW+2.1
footprint_ei39 = 'data/IW+2.1/impact_world_plus_21_brightway2_footprint_version_ei39.fcfdfe7f6cab324019dc181525db2f39.bw2package'
bw.BW2Package.import_file(footprint_ei39)

In [ ]:
# Filter and display methods that contain "IMPACT World+" in their names
iw_methods = [method for method in bd.methods if "impact world+" in " ".join(method).lower()]
df_iw_methods = pd.DataFrame(iw_methods, columns=["Method", "Impact Category", "Subcategory"])
#df_iw_methods.to_csv(r'data/iw_methods.csv', index=False)

In [ ]:
# To take them all
IMPACT_METHODS_ALL = {method[-1]: method for method in iw_methods}
IMPACT_METHODS_ALL

In [ ]:
# We take only a few one for testing
IMPACT_METHODS = {
'Climate change ST': ('IMPACT World+ Midpoint 2.0.1_regionalized','Midpoint', 'Climate change, short term'), 
'Total human health': ('IMPACT World+ Damage 2.0.1_regionalized','Human health', 'Total human health'), 
'Total ecosystem quality': ('IMPACT World+ Damage 2.0.1_regionalized','Ecosystem quality', 'Total ecosystem quality'), 
}


# Calculate specific impacts (e.g. per mass)

In [ ]:
# Initialize LCA object
lca = init_simple_lca(INVENTORIES_ds["Lithium hydroxide, brine"], method=IMPACT_METHODS)

In [ ]:
specific_lca = {}
for rm in INVENTORIES_ds:
    impacts = multi_lcia(lca, INVENTORIES_ds[rm], IMPACT_METHODS)
    specific_lca[rm] = impacts

specific_lca = pd.DataFrame(specific_lca).T

In [ ]:
specific_lca

In [ ]:
# Call the function
plot_multilca_impacts(specific_lca, 
                 colors=["#1f77b4", "#ff7f0e", "#2ca02c"], 
                 save_path="results/specific_lca_results/lca_impacts_iw201_reg.png")

In [ ]:
specific_lca.to_csv(f"results/specific_lca_results/specific_results_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

In [ ]:
# Initialize a dictionary to store contribution results for each inventory
contribution_results = {}

# Loop through each inventory and perform contribution analysis
for rm_name, rm_ds in INVENTORIES_ds.items():
    lca = init_simple_lca(rm_ds)
    contributions = multi_contribution_analysis(lca, IMPACT_METHODS, top_n=10, threshold=0.01)  # Set your threshold here
    contribution_results[rm_name] = contributions

# Convert the results into a more readable format for analysis
contribution_dfs = {}

for rm_name, impacts in contribution_results.items():
    for impact_name, contributions in impacts.items():
        df = pd.DataFrame(contributions)
        df["Inventory"] = rm_name
        df["Impact Category"] = impact_name
        contribution_dfs[(rm_name, impact_name)] = df

# Combine all individual DataFrames into one for easy viewing
contribution_analysis_df = pd.concat(contribution_dfs.values(), ignore_index=True)

# Set "Inventory" and "Impact Category" as the row indices
contribution_analysis_df.set_index(["Inventory", "Impact Category"], inplace=True)


In [ ]:
contribution_analysis_df

In [ ]:
# Example usage
inventory_names = ["Lithium hydroxide, brine", 
                   "Lithium hydroxide, spodumene", 
                   "Cobalt", 
                   "Nickel", 
                   "Graphite, natural", 
                   "Graphite, synthetic", 
                   "Lithium carbonate, brine", 
                   "Lithium carbonate, spodumene",
                   "Neodymium", 
                   "Copper concentrate, Canada", 
                   "Market for copper, cathode"           
                   ]
plot_contribution_analysis(contribution_analysis_df, inventory_names,
                           colors=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"], 
                           save_dir="results/specific_lca_results/contribution_analysis")

In [ ]:
contribution_analysis_df.to_csv(f"results/specific_lca_results/contribution_analysis/contribution_analysis_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

## See causes of HH and EQ 

We compute all subcategories for ecosystem quality and human health to see their relative contributions to their respective total impacts

In [ ]:
# Filter to remove midpoint indicators and footprint entries
IMPACT_METHODS_DAMAGE = {
    key: value for key, value in IMPACT_METHODS_ALL.items()
    if value[1] != "Midpoint" and value[0] != "IMPACT World+ Footprint 2.1 for ecoinvent v3.10"
}

In [ ]:
IMPACT_METHODS_DAMAGE

In [ ]:
 #Initialize LCA object
lca_damage = init_simple_lca(INVENTORIES_ds["Lithium hydroxide, brine"], method=IMPACT_METHODS_DAMAGE)

In [ ]:
specific_lca_damage = {}
for rm in INVENTORIES_ds:
    impacts = multi_lcia(lca_damage, INVENTORIES_ds[rm], IMPACT_METHODS_DAMAGE)
    specific_lca_damage[rm] = impacts

specific_lca_damage = pd.DataFrame(specific_lca_damage).T

In [ ]:
specific_lca_damage

In [ ]:
specific_lca_damage.to_csv(f"results/specific_lca_results/specific_results_hh_eq{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index_label="Raw material")

In [ ]:
plot_iwplus_contributions(specific_lca_damage, save_path_eco="results/specific_lca_results/ecosystem_quality_contributions.png", save_path_hh="results/specific_lca_results/human_health_contributions.png")

# Calculate production-related impacts (e.g. scaled with scenarios)

## Import scenarios

They are from the Canadian Climate Institute and can be found [here](https://440megatonnes.ca/insight/canada-critical-minerals-clean-energy-transition/)

In [ ]:
production_existing = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_existing')
production_potential = pd.read_excel(r'data/scenarios_canadian_climate_institute.xlsx', sheet_name='Production_potential')
production_existing = production_existing[production_existing['Scenario']=='Domestic demand scenario']
production_potential = production_potential[production_potential['Scenario']=='Domestic demand scenario']

In [ ]:
plot_scenario_production_comparison(production_existing, production_potential, save_path='results/demand_lca_results/cci_production_scenarios.png')

## Choose which LCI to associate with each mineral 

In [ ]:
specific_lca = specific_lca.reset_index().rename(columns={'index': 'Mineral'})
specific_lca

In [ ]:
# Define a manual mapping to link the mineral in the scenario with the LCI of our choice
mineral_to_material = {
    "Cobalt": "Cobalt",
    "Copper": "Copper concentrate, Canada",
    "Lithium": "Lithium carbonate, spodumene", 
    "Nickel": "Nickel",
    "Graphite": "Graphite, natural",
    "Neodymium": "Neodymium"  
}


## Calculate the production-related impacts of scenarios

In [ ]:
# Generate the projected impacts DataFrame
projected_impacts_existing_production = calculate_projected_impacts(production_existing, specific_lca, mineral_to_material)
projected_impacts_potential_production = calculate_projected_impacts(production_potential, specific_lca, mineral_to_material)


In [ ]:
projected_impacts_existing_production

In [ ]:
projected_impacts_existing_production.columns

In [ ]:
impact_categories = [
'Climate change ST (kg CO2 eq (short))',
'Total human health (DALY)',
'Total ecosystem quality (PDF.m2.yr)'    
]

In [ ]:
# Plot impacts per production scenario
plot_production_impacts(projected_impacts_existing_production, production_existing, 
                             impact_categories, 
                             scenario_name='existing_production')
plot_production_impacts(projected_impacts_potential_production, production_potential, 
                             impact_categories, scenario_name='potential_production')


In [ ]:
# Plot incremental impacts from potential production to existing production
plot_incremental_impacts(projected_impacts_existing_production, projected_impacts_potential_production,
                                 production_existing, production_potential, 
                                 impact_categories, save_dir="results/demand_lca_results", scenario_name="incremental_comparison")
